In [107]:
from google.transit import gtfs_realtime_pb2
import requests
import plotly.express as px
import pandas as pd
import json
import time
import math
import numpy as np

pd.options.display.width = 0

In [108]:
def extract_entity_data(entity):
    return {
        'insert_ts': int(time.time()),
        'vehicle_update_ts': entity.vehicle.timestamp,
        'lat': entity.vehicle.position.latitude,
        'lon': entity.vehicle.position.longitude,
        'trip': entity.vehicle.trip.trip_id,
        'route_id': entity.vehicle.trip.route_id,
        'stop_id': entity.vehicle.stop_id,
        'vehicle_id': entity.vehicle.vehicle.id,
        'vehicle_label': entity.vehicle.vehicle.label
    }

def haversine_distance_to_me(lat1, lon1, lat2=39.75342747354773, lon2=-105.0009645548709):
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    lat1 = lat1 * math.pi / 180.0
    lat2 = lat2 * math.pi / 180.0

    a = np.sin(dLat / 2) ** 2 + np.sin(dLon / 2) ** 2 * np.cos(lat1) * np.cos(lat2)
    rad = 6371
    c = 2 * np.arcsin(np.sqrt(a))
    return rad * c

def refresh_vehicles():
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get('https://www.rtd-denver.com/files/gtfs-rt/VehiclePosition.pb')
    feed.ParseFromString(response.content)

    records = []
    for entity in feed.entity:
        records.append(extract_entity_data(entity))

    rt_positions = pd.DataFrame.from_records(records)

    backyard_routes = ['113G', '113B', '117N', 'A']

    relevant_vehicles = rt_positions[rt_positions.route_id.isin(backyard_routes)].copy()

    relevant_vehicles['distance'] = haversine_distance_to_me(relevant_vehicles['lat'], relevant_vehicles['lon'])

    relevant_vehicles['insert_ts_seconds_ago'] = int(time.time()) - relevant_vehicles['insert_ts'].astype(int)
    relevant_vehicles['vehicle_update_seconds_ago'] = int(time.time()) - relevant_vehicles['vehicle_update_ts'].astype(int)

    return relevant_vehicles

In [110]:
old_positions = refresh_vehicles()

while True:
    new_positions = refresh_vehicles()

    position_comparison = new_positions[['vehicle_label', 'route_id', 'distance', 'vehicle_update_seconds_ago']].merge(old_positions[['vehicle_label', 'distance', 'vehicle_update_seconds_ago']], on='vehicle_label', suffixes=['_new', '_old'], how='left').fillna(9999)
    position_comparison['status'] = None
    position_comparison.loc[(position_comparison['distance_new'] < 0.2) & (position_comparison['distance_old'] > 0.2), 'status'] = 'arriving'
    position_comparison.loc[(position_comparison['distance_new'] < 0.2) & (position_comparison['distance_old'] < 0.2), 'status'] = 'at station'
    position_comparison.loc[(position_comparison['distance_new'] > 0.2) & (position_comparison['distance_old'] < 0.2), 'status'] = 'departing'
    position_comparison.loc[(position_comparison['distance_new'] > 0.2) & (position_comparison['distance_old'] > 0.2), 'status'] = 'away'

    updates = position_comparison[position_comparison['status'].isin(['arriving', 'departing', 'at station'])]
    if len(updates) > 0:
        print(time.time())
        print(updates)

    else:
        print('no updates')
        print(position_comparison)
    
    old_positions = new_positions

    time.sleep(60)

1721448634.903899
  vehicle_label route_id  distance_new  vehicle_update_seconds_ago_new  \
7     4015,4016     113B      0.184955                              95   

   distance_old  vehicle_update_seconds_ago_old      status  
7      0.184955                              95  at station  
1721448695.7039013
  vehicle_label route_id  distance_new  vehicle_update_seconds_ago_new  \
7     4015,4016     113B      0.184955                             156   

   distance_old  vehicle_update_seconds_ago_old      status  
7      0.184955                              95  at station  
1721448756.2513063
  vehicle_label route_id  distance_new  vehicle_update_seconds_ago_new  \
7     4015,4016     113B      0.184955                             217   

   distance_old  vehicle_update_seconds_ago_old      status  
7      0.184955                             156  at station  
1721448816.760507
  vehicle_label route_id  distance_new  vehicle_update_seconds_ago_new  \
7     4015,4016     113B      0.7

KeyboardInterrupt: 

In [103]:
refresh_vehicles()

,insert_ts,vehicle_update_ts,lat,lon,trip,route_id,stop_id,vehicle_id,vehicle_label,distance,insert_ts_seconds_ago,vehicle_update_seconds_ago
166,1721447390,1721447188,39.768715,-104.866081,114912783,A,34473,1D979E9D3BA64538E063DD4D1FACE778,"4025,4026,4035,4036",11.654137,0,202
167,1721447390,1721447154,39.803993,-105.023941,114900992,113G,34526,1D98110D596FA0EEE063DD4D1FAC4B33,"4007,4008",5.955625,0,236
168,1721447390,1721447164,39.766647,-104.837769,114912908,A,34470,1D982681E5E2181BE063DD4D1FAC5784,"4029,4030,4059,4060",14.027042,0,226
169,1721447390,1721447043,39.772976,-104.996994,114900967,113G,34781,1D9875305BDBE56DE063DD4D1FACAA3F,"4031,4032",2.200023,0,347
170,1721447390,1721447188,39.902512,-104.960197,114901493,117N,35252,1D98D5C654DC601AE063DD4D1FACA614,"4045,4046",16.938992,0,202
171,1721447390,1721447192,39.838036,-104.947861,114901447,117N,35251,1D992F2E819A5515E063DD4D1FAC8713,"4063,4064",10.444797,0,198
172,1721447390,1721447140,39.814045,-105.014275,114900932,113B,34543,1D992F2E81A45515E063DD4D1FAC8713,"4015,4016",6.835637,0,250
